In [66]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers
from utils.index_helper import IndexHelper

In [ ]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [ ]:
clustered_ratings_df = pd.read_csv('../data/produced/clustered_ratings.csv')
clustered_ratings_df

In [ ]:
unique_clusters = len(clustered_ratings_df['cluster'].unique())
print(f'Unique clusters: {unique_clusters}')
unique_isbns = len(clustered_ratings_df['isbn'].unique())
print(f'Unique ISBNs: {unique_isbns}')

In [ ]:
def index_data(records: {}, index: str):
    for record in records:
        yield {
            "_index": index,
            "_source": record
        }

In [ ]:
clustered_ratings_dict = clustered_ratings_df.to_dict('records')
helpers.bulk(es, index_data(clustered_ratings_dict, 'clustered_ratings'))

In [ ]:
response = es.search(
    index="clustered_ratings",
    aggs={
        "cluster_agg": {
            "terms": {
                "field": "cluster",
                "size": unique_clusters
            },
            "aggs": {
                "isbn_agg": {
                    "terms": {
                        "field": "isbn.keyword",
                        "size": unique_isbns
                    },
                    "aggs": {
                        "avg_rating": {
                            "avg": {
                                "field": "rating"
                            }
                        }
                    }
                }
            }
        }
    })

In [63]:
avg_ratings_df = pd.DataFrame([{
    "cluster": cluster_bucket['key'],
    "isbn": isbn_bucket['key'],
    "avg_rating": isbn_bucket['avg_rating']['value']
} for cluster_bucket in response['aggregations']['cluster_agg']['buckets']
    for isbn_bucket in cluster_bucket['isbn_agg']['buckets']])

avg_ratings_df['avg_rating'] = avg_ratings_df['avg_rating'].round(2)

avg_ratings_df

,cluster,isbn,avg_rating
0,1,0971880107,1.28
1,1,044023722X,4.70
2,1,0312195516,6.25
3,1,0060928336,4.87
4,1,0452282152,5.25
...,...,...,...
28216,5,1842550942,8.00
28217,5,1853260495,0.00
28218,5,1853260622,8.00
28219,5,1892958015,0.00


In [ ]:
avg_ratings_df.to_csv('../data/produced/avg_ratings.csv', index=False)

In [67]:
ih = IndexHelper(es)
ih.bulk_index(data=avg_ratings_df, index='avg_ratings')

C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\es_index.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if self.es.indices.exists(index=index):
C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\es_index.py:22: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  self.es.indices.delete(index=index)


Deleting index...


C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\es_index.py:25: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(self.es, _index_data(data_dict, index))
